In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os

from keras.layers import Concatenate, Input, Dense, Embedding, Flatten, Dropout, BatchNormalization, SpatialDropout1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import  Adam
import keras.backend as k

# import matplotlib.pyplot as plt
# %matplotlib inline
# from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
# from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
# pd.options.display.precision = 15
from category_encoders.cat_boost import CatBoostEncoder

# import lightgbm as lgb
# import xgboost as xgb
# import time
# import datetime
# from catboost import CatBoostRegressor
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
# from sklearn import metrics
# from sklearn import linear_model
import gc
import pickle
# import seaborn as sns
# import warnings
# warnings.filterwarnings("ignore")

# import eli5
# import shap
# from IPython.display import HTML
# import json
# import altair as alt

# import networkx as nx
# import matplotlib.pyplot as plt

gc.collect()
# alt.renderers.enable('notebook')

Using TensorFlow backend.
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

0

In [2]:
main_path = r'd:\Documents\Private\Kaggle\IEEEFraud'

In [3]:
import sys
sys.path.append(main_path)
from BayDS import *

In [4]:
experiment_name = '31.08'
main_learning_folder = main_path+'/Snapshots/'+experiment_name


In [5]:
train = pd.read_pickle(f'{main_learning_folder}/train.pkl')
test = pd.read_pickle(f'{main_learning_folder}/test.pkl')


In [6]:
X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT'], axis=1)
y = train.sort_values('TransactionDT')['isFraud'].astype(np.uint8)
test = test.sort_values('TransactionDT').drop(['TransactionDT'], axis=1)

X.drop('Date', axis=1, inplace=True)
X=X.astype(np.float32)
test.drop('Date', axis=1, inplace=True)
test=test.astype(np.float32)
del train

In [7]:
gc.collect()

40

In [8]:
cat_anya = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15',
       'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33',
       'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo',
       'P_emaildomain_bin', 'P_emaildomain_suffix', 'R_emaildomain_bin',
       'R_emaildomain_suffix', 'device_name', 'device_version', 'OS_id_30',
       'version_id_30', 'browser_id_31', 'version_id_31', 'screen_width',
       'screen_height', 'uid', 'uid2', 'uid3']

In [9]:
cat_vasya = ["id_13", "id_14", "id_17", "id_18", "id_19", "id_20", "id_21", "id_22", "id_23", 
 "id_24", "id_25", "id_26", "id_27", "id_30", "id_31", "id_33", "DeviceType", 
 "DeviceInfo", "P_emaildomain", "R_emaildomain", "card1", "card2", "card3", 
 "card5", "addr1", "addr2", "P_emaildomain_bin", "P_emaildomain_suffix", 
 "R_emaildomain_bin", "R_emaildomain_suffix", "M1", "M2", "M3", "M4", "M5", 
 "M6", "M7", "M8", "M9", "card4", "card6", "ProductCD", "id_12", "id_15", 
 "id_16", "id_28", "id_29", "id_32", "id_34", "id_35", "id_36", "id_37", "id_38"]

In [10]:
cat_features = list(set(cat_vasya) | set(cat_anya))

In [11]:
merg = pd.concat([X,test])


In [12]:
encoder = CatBoostEncoder(verbose=1,cols=cat_features)
encoder.fit(X,y)


CatBoostEncoder(cols=['M3', 'id_17', 'R_emaildomain', 'screen_height', 'id_19',
                      'id_26', 'card6', 'OS_id_30', 'id_27', 'id_13',
                      'R_emaildomain_bin', 'id_22', 'card2', 'id_21', 'uid',
                      'id_28', 'uid2', 'id_12', 'addr1', 'card3', 'id_32',
                      'DeviceInfo', 'id_36', 'M7', 'version_id_30', 'id_35',
                      'M1', 'M8', 'id_14', 'card1', ...],
                drop_invariant=False, handle_missing='value',
                handle_unknown='value', random_state=None, return_df=True,
                sigma=None, verbose=1)

In [13]:
del X,test

In [14]:
gc.collect()
merg = encoder.transform(merg)

In [15]:
merg.to_pickle(f'{main_learning_folder}/catboostencoder.pkl')

